In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pickle

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [28]:
def extract_data():
    train_data=[]
    for i in range(1,6):
        file='./cifar-10-batches-py/data_batch_'+str(i)
        temp=unpickle(file)
        temp_data=[]
        temp_data.append(temp[b'data'].astype('float64'))
        temp_data.append(np.asarray(temp[b'labels']).reshape(temp[b'data'].shape[0],1))
        train_data.append(temp_data)
    temp=unpickle('./cifar-10-batches-py/test_batch')
    temp_data=[]
    temp_data.append(temp[b'data'].astype('float64'))
    temp_data.append(np.asarray(temp[b'labels']).reshape(temp[b'data'].shape[0],1))
    test_data=temp_data
    temp=unpickle('./cifar-10-batches-py/batches.meta')
    labels=temp[b'label_names']
    for i in range(len(labels)):
        labels[i]=(labels[i].decode("utf-8"))
    return train_data,test_data,labels



In [42]:
def normalize(data):
    mean = np.mean(data, axis = 0)
    data -= mean
    return data
def svm_linear(data,test):
    data[0]=data[0].astype('float64')
    test[0]=test[0].astype('float64')
    data[0]=normalize(data[0])
    test[0]=normalize(test[0])
    X=data[0]
    Y=data[1]
    clf = SVC(kernel='linear', C=10, gamma=0.01,decision_function_shape='ovr',verbose=True,max_iter=100)
    clf.fit(X,Y)
    accuracy = clf.score(test[0],test[1])
    print ("Accuracy: ", accuracy)

In [43]:
train_data,test_data,labels=extract_data()
svm_linear(train_data[0],test_data)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Accuracy:  0.2205
